In [4]:
import os
import json
import pandas as pd
import requests
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', -1)

<ipython-input-4-a6c9beac1ab0>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [5]:
def getDataframe(filename):
    arr = []
    with open('./data/' + filename) as out:
        for line in out:
            j_content = json.loads(line)
            arr.append(j_content)
    df = pd.DataFrame(arr)
    return df

In [6]:
def mongoIdToId(value):
    return dict(value)["$oid"]

In [7]:
producers = getDataframe('producers.json')
producers["organization_id"] = producers["associatedTo"].apply(mongoIdToId)
producers.head()

,_id,associatedTo,__v,organization_id
0,AFB,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
1,AFB - DR Nouvelle-Aquitaine,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
2,ALTOA,{'$oid': '580fb83a88ee38771813e4cb'},NaN,580fb83a88ee38771813e4cb
3,ANFR,{'$oid': '54256fe588ee380af6717348'},NaN,54256fe588ee380af6717348
4,ARS,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123


In [8]:
organizations = getDataframe('organizations.json')
organizations["geop_organization_id"] = organizations["_id"].apply(mongoIdToId)
organizations = organizations.merge(producers, left_on="geop_organization_id", right_on="organization_id")
organizations["geop_organization_name"] = organizations["_id_y"]
organizations = organizations[["geop_organization_id", "geop_organization_name"]]
organizations.head()

,geop_organization_id,geop_organization_name
0,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire
1,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire (71)
2,534fff6aa3a7292c64a77d96,Région Hauts-de-France
3,534fff6ba3a7292c64a77da1,ASPA
4,534fff6ba3a7292c64a77da1,CConseil Départemental du Bas-Rhin (CD67)


In [9]:
organizations['dgv_organization_id'] = organizations['geop_organization_id']

In [10]:
dgv_catalog_orgas = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/b7bbfedc-2448-4135-a6c7-104548d396e7',sep=';')

In [11]:
dgv_catalog_orgas = dgv_catalog_orgas[['id','name', 'metric.datasets']]
dgv_catalog_orgas.head()

,id,name,metric.datasets
0,62963d5fcfba8babf5842846,COMMUNE DE LAMANON,0
1,62960bfebb22e50019faa2a3,Service SIG - Ville d'Aix en Provence,1
2,62960a74e4d4fc681ae0c9ad,DOCTEUR MONNOT THIONVILLE BORNE DE CHARGE,1
3,6295de6ec5ab58d9001ac8b0,Département de la Creuse,0
4,6295ba80cdbed53704d24f60,AboSociete - Service par Abonnement B2B,0


In [12]:
dgv_catalog_orgas = dgv_catalog_orgas.rename(columns={
    'id': 'dgv_organization_id',
    'name': 'dgv_organization_name', 
    'metric.datasets': 'dgv_nb_datasets',
})

In [13]:
# no duplicates
assert organizations[organizations.duplicated()].empty
assert dgv_catalog_orgas[dgv_catalog_orgas.duplicated()].empty

In [14]:
organizations.columns

Index(['geop_organization_id', 'geop_organization_name',
       'dgv_organization_id'],
      dtype='object')

In [15]:
dgv_catalog_orgas.dgv_organization_id.nunique()

3428

In [16]:
# outer merge to pivot on dgv orgas, not necessarily on geop orgs
final_orgas = dgv_catalog_orgas.merge(organizations, on='dgv_organization_id', how='outer')
assert final_orgas[final_orgas.duplicated()].empty
final_orgas.head()

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,geop_organization_id,geop_organization_name
0,62963d5fcfba8babf5842846,COMMUNE DE LAMANON,0.0,NaN,NaN
1,62960bfebb22e50019faa2a3,Service SIG - Ville d'Aix en Provence,1.0,NaN,NaN
2,62960a74e4d4fc681ae0c9ad,DOCTEUR MONNOT THIONVILLE BORNE DE CHARGE,1.0,NaN,NaN
3,6295de6ec5ab58d9001ac8b0,Département de la Creuse,0.0,NaN,NaN
4,6295ba80cdbed53704d24f60,AboSociete - Service par Abonnement B2B,0.0,NaN,NaN


In [17]:
# geoc = pd.read_excel('./data/Liste_catalogues_DCAT.xlsx')
geoc = pd.read_excel('./data/Liste_producteur_fevrier_avec_ressources.xlsx')
del geoc["Unnamed: 3"]

In [18]:
geoc = geoc.rename(columns={
    'Producteurs': 'geoc_organization_name', 
    'uuid': 'geoc_organization_uuid', 
    'url de moissonnage ': 'geoc_organization_url'
})
geoc.head()

,geoc_organization_name,geoc_organization_uuid,geoc_organization_url
0,Agence de l'eau Adour Garonne (AEAG),bdb31aaf37e0f29db345d2b62d7ccccb,http://www.geocatalogue.fr/www/dcat/bdb31aaf37e0f29db345d2b62d7ccccb
1,"Agence nationale de sécurité sanitaire de l'alimentation, de l'environnement et du travail (ANSES)",cadef4c11943e372e8afd0b94797b5ec,http://www.geocatalogue.fr/www/dcat/cadef4c11943e372e8afd0b94797b5ec
2,Agence Technique de l'information sur l'hospitalisation (ATIH),98283bf812e489423f273f61a4e4e431,http://www.geocatalogue.fr/www/dcat/98283bf812e489423f273f61a4e4e431
3,Agglomération du Choletais,e05f395b2c2b3a94fabd65275d6ae266,http://www.geocatalogue.fr/www/dcat/e05f395b2c2b3a94fabd65275d6ae266
4,Air Pays de la Loire,f540ee38176857fd4ed78c5c321b9ac0,http://www.geocatalogue.fr/www/dcat/f540ee38176857fd4ed78c5c321b9ac0


In [19]:
def removeCommonWords(org):
    org = str(org)
    org = org.replace(' de ',' ').replace(' un ',' ').replace(' à ','').replace(' l\'',' ').replace(' d\'', ' ')
    org = org.replace(' des ',' ').replace(' pour ', ' ').replace(' les ', ' ').replace('(','').replace(')','')
    org = org.replace(' - ',' ').replace(',','').replace(' du ', ' ').replace('à','a').replace('é','e')
    org = org.replace('ê','e').replace('è','e').replace('\'',' ')
    return org.lower()

In [20]:
geoc["geoc_slug"] = geoc["geoc_organization_name"].apply(removeCommonWords)
final_orgas["geop_slug"] = final_orgas["geop_organization_name"].apply(removeCommonWords)
final_orgas["dgv_slug"] = final_orgas["dgv_organization_name"].apply(removeCommonWords)

In [21]:
df_dgv_match = final_orgas.merge(geoc, left_on="dgv_slug", right_on="geoc_slug")
df_dgv_match["geoc_organization_url"].drop_duplicates().shape

(99,)

In [22]:
df_geop_match = final_orgas.merge(geoc, left_on="geop_slug", right_on="geoc_slug")
df_geop_match["geoc_organization_url"].drop_duplicates().shape

(33,)

In [23]:
df_final = df_dgv_match.merge(df_geop_match, how="outer")
df_final[df_final['geoc_organization_uuid'].notna()]["geoc_organization_url"].drop_duplicates().shape

(122,)

In [24]:
df_final[df_final['geoc_organization_uuid'].notna()].head()

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,geop_organization_id,geop_organization_name,geop_slug,dgv_slug,geoc_organization_name,geoc_organization_uuid,geoc_organization_url,geoc_slug
0,627e6aa33a3874a12a844a62,Communauté d'Agglomération du Grand Villeneuvois,0.0,NaN,NaN,nan,communaute agglomeration grand villeneuvois,Communauté d'Agglomération du Grand Villeneuvois,49b16c22f3e4956b8fae35eace1d34d2,http://www.geocatalogue.fr/www/dcat/49b16c22f3e4956b8fae35eace1d34d2,communaute agglomeration grand villeneuvois
1,62287c5b201dffeb66cc385a,BRL,3.0,NaN,NaN,nan,brl,BRL,1a0b4aa361bb5315e304831a320f89b3,http://www.geocatalogue.fr/www/dcat/1a0b4aa361bb5315e304831a320f89b3,brl
2,61fc10b5285e11c883d018bd,MAIRIE LES TROIS DOMAINES,0.0,NaN,NaN,nan,mairie les trois domaines,Mairie de Les Trois domaines,6fc21c45008161fcadd99a9c53e0351f,http://www.geocatalogue.fr/www/dcat/6fc21c45008161fcadd99a9c53e0351f,mairie les trois domaines
3,61b0b0673c11d387da1df057,Pontivy Communaute,0.0,NaN,NaN,nan,pontivy communaute,Pontivy Communauté,eac49cc9bdb321a31f6150862adc28b7,http://www.geocatalogue.fr/www/dcat/eac49cc9bdb321a31f6150862adc28b7,pontivy communaute
4,61484aaaffc949c8b4971434,Préfecture de la région Hauts-de-France,1.0,NaN,NaN,nan,prefecture la region hauts-de-france,Préfecture de la région Hauts-de-France,0709d43ca86491e10b8d95b9ae7ad916,http://www.geocatalogue.fr/www/dcat/0709d43ca86491e10b8d95b9ae7ad916,prefecture la region hauts-de-france


In [25]:
df_final.to_csv('mapping_geop_dgv_geoc-pivot-datagouv-v2.csv', index=False)

In [26]:
df_final.shape

(173, 11)

In [38]:
# GEO-IDE

df_geoide = pd.read_excel("Geocatalogue_producteurs Geo-IDE.xlsx")
df_geoide["geoide_slug"] = df_geoide["producteur"].apply(removeCommonWords)
dgv_catalog_orgas["dgv_slug"] = dgv_catalog_orgas["dgv_organization_name"].apply(removeCommonWords)
df_geoide_match = dgv_catalog_orgas.merge(df_geoide, left_on="dgv_slug", right_on="geoide_slug")
df_geoide_match

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,dgv_slug,adh_name,producteur,nb_MD,geoide_slug


In [60]:
def fuzzy_search(org):
    # url = "https://www.data.gouv.fr/api/1/organizations?q={}"
    url = "https://www.data.gouv.fr/api/2/organizations/search/?q={}"
    if len(org) <= 3:
        return None, None
    org_vector = org.replace("(", "").replace(")", "").replace("-", " ")
    org_vector = org_vector.split(" ")
    q = ""
    result = None, None
    for part in org_vector:
        q += f" {part}"
        if len(q) <= 3:
            continue
        r = requests.get(url.format(q))
        data = r.json()
        if data['total'] == 0:
            print(f'No result for {q}')
            break
        elif data['total'] == 1:
            result = data['data'][0]['id'], data['data'][0]['name']
            print(f"Found {data['data'][0]['name']} for {q}")
            break
        else:
            pass
            # print(f"Iterating, found {data['total']} for {q}...")
    return result

results = df_geoide["producteur"].apply(fuzzy_search)

No result for  Atelier de Suivi
Found Cerema for  Centre d'études et d'Expertise
No result for  COVADIS
Found DDT de l'Ain for  DDT 01
Found DDT Aisne for  DDT 02
Found Direction Départementale des Territoires de l'Allier for  DDT 03
Found Direction Départementale des Territoires des Alpes de Haute-Provence for  DDT 04
No result for  DDT 05 Direction
Found DDT de l'Ardèche for  DDT 07
Found DDT Ardennes for  DDT 08
Found Direction Départementale des Territoires de l'Ariège for  DDT 09
Found Direction Départementale des Territoires de l'Aube for  DDT 10
Found DDT Aveyron for  DDT 12
Found Direction Départementale et des Territoires du Cantal for  DDT 15
Found Direction Départementale des Territoires de Charente for  DDT 16
Found Direction Départementale des Territoires du Cher for  DDT 18
Found Direction Départementale des Territoires de la Corrèze for  DDT 19
Found DDT Côte-d'Or for  DDT 21
Found DDT Creuse for  DDT 23
Found DDT Dordogne for  DDT 24
Found Direction Départementale des T

No result for  SPRS


In [77]:
df_geoide[["dgv_match_id", "dgv_match_name"]] = pd.DataFrame(results.tolist(), index=df_geoide.index)
df_geoide.to_csv('geoide-datagouv-mapping.csv',index=False)

In [25]:
# ENRICH DATA with api calls
from collections import defaultdict

url = "https://www.data.gouv.fr/api/1"
auth = {
    "x-api-key": os.getenv("DATAGOUV_TOKEN")
}

def get_org_data(org_id):
    org_data = defaultdict(list)
    org_data["id"] = org_id
    r = requests.get(f"{url}/organizations/{org_id}/")
    data = r.json()

    # members
    for m in data["members"]:
        r = requests.get(m["user"]["uri"], headers=auth)
        mdata = r.json()
        org_data[m["role"]].append(f"{mdata['first_name']} {mdata['last_name']} <{mdata['email']}>")
    
    org_data["admin"] = ", ".join(org_data["admin"])
    org_data["editor"] = ", ".join(org_data["editor"])

    # datasets
    r = requests.get(f"{url}/organizations/{org_id}/datasets/?page_size=1000")
    datasets = r.json()
    assert datasets["total"] == len(datasets["data"])
    org_data["nb_datasets"] = datasets["total"]
    org_data["nb_datasets_geop"] = len([d["id"] for d in datasets["data"] if "geop:dataset_id" in d["extras"]])
    org_data["last_dataset_modified_date"] = max([d["last_modified"] for d in datasets["data"]]) if datasets["data"] else None
        
    return dict(org_data)

In [26]:
metadata = []
for org_id in df_final["dgv_organization_id"].drop_duplicates():
    metadata.append(get_org_data(org_id))

In [27]:
del df_final["dgv_nb_datasets"]

In [28]:
r = requests.get(f"{url}/harvest/sources/?page_size=1000", headers=auth)
harvesters = r.json()
assert harvesters["total"] == len(harvesters["data"])
len(harvesters["data"])

277

In [66]:
# harvesters
def get_harvest_data(org_id):
    hs = [h for h in harvesters["data"] if h["organization"] and h["organization"].get("id") == org_id]
    return {
        "id": org_id,
        "nb_harvesters": len(hs),
        "harvesters": " | ".join([
            f"{h['id']} — {h['name']} — {h['url']}" for h in hs
        ])
    }

harvesters_match = []
for org_id in df_final["dgv_organization_id"].drop_duplicates():
    harvesters_match.append(get_harvest_data(org_id))
    
len([h for h in harvesters_match if h['nb_harvesters']])

20

In [29]:
meta_df = pd.DataFrame(metadata)
harvest_df = pd.DataFrame(harvesters_match)
df_meta = df_final.merge(meta_df, right_on="id", left_on="dgv_organization_id")
del df_meta["id"]
df_meta = df_meta.merge(harvest_df, right_on="id", left_on="dgv_organization_id")
del df_meta["id"]
df_meta

NameError: name 'harvesters_match' is not defined

In [68]:
df_meta.to_csv('mapping_geop_dgv_geoc-pivot-datagouv_w_metadata_harvest.csv',index=False)

In [26]:
# TODO:
# - [x] add nb of datasets
# - [x] add nb of geop datasets
# - [x] add admin contacts
# - [x] add members contacts
# - [x] last modified dataset date
# - [x] match w/o joining on geop_organizations (dgv_organizations)